```
sudo apt-get install portaudio19-dev
sudo apt-get install ffmpeg
pip install sounddevice
pip install git+https://github.com/openai/whisper.git
pip install wave
```

In [ ]:
import sounddevice as sd
import numpy as np
import whisper
import wave
import io
import time
import threading
import tempfile

RATE = 16000
DURATION = 5
CHANNELS = 1
recording_data = []

# Chargement du modèle Whisper
model = whisper.load_model("large")  # le modèle small est le plus rapide

def record_audio(duration, rate, channels):
    global recording_data
    recording_data = []
    print("Enregistrement en cours...")

    with sd.InputStream(samplerate=rate, channels=channels, dtype='int16') as stream:
        while True:
            data, overflowed = stream.read(1024)
            recording_data.append(data)
            time.sleep(0.1)
            if not recording:
                break

    print("Enregistrement arrêté.")

def stop_recording():
    global recording
    recording = False

def audio_to_wav(audio_data, rate):
    with io.BytesIO() as audio_file:
        with wave.open(audio_file, 'wb') as wf:
            wf.setnchannels(1)
            wf.setsampwidth(2)
            wf.setframerate(rate)
            wf.writeframes(b''.join(audio_data))
        return audio_file.getvalue()

def transcribe_audio(audio_data, rate):
    wav_audio = audio_to_wav(audio_data, rate)

    with tempfile.NamedTemporaryFile(suffix=".wav", delete=True) as tmp:
        tmp.write(wav_audio)
        tmp.flush()

        result = model.transcribe(tmp.name)

    return result["text"]

def start_stop_recording():
    global recording
    recording = True

    record_thread = threading.Thread(target=record_audio, args=(DURATION, RATE, CHANNELS))
    record_thread.start()

    time.sleep(DURATION)
    stop_recording()

    combined_audio = np.concatenate(recording_data)

    # Transcription
    transcription = transcribe_audio(combined_audio, RATE)
    print("Transcription :")
    print(transcription)

start_stop_recording()
